In [24]:
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import copy
import geopandas as gpd  # Import geopandas
import fiona #공간데이터를 딕셔너리 형태 등으로 접근할 수 있는 라이브러리
import pyproj
from fiona.crs import from_epsg
from scipy import stats
from sklearn.model_selection import train_test_split


os.chdir('C:/Users/user/Desktop/서울시청/데이터')
# 폰트 설정
mpl.rc('font', family=' Malgun Gothic')
# 유니코드에서  음수 부호설정
mpl.rc('axes', unicode_minus=False)

warnings.filterwarnings('ignore')

In [6]:
place=pd.read_csv('공공자전거 대여소 정보(20.07.13 기준).csv',encoding='cp949')

In [8]:
place.head()

,대여소번호,보관소(대여소)명,자치구,상세주소,위도,경도,설치시기,LCD(거치대수),QR(거치대수),운영방식
0,1695,월계미륭아파트 정문,노원구,노원구 월계동 14,37.623417,127.066933,2020-06-17,NaN,10.0,QR
1,2301,현대고등학교 건너편,강남구,서울특별시 강남구 압구정로 134,37.524071,127.021790,2017-06-13,10.0,NaN,LCD
2,2302,교보타워 버스정류장(신논현역 3번출구 후면),강남구,서울특별시 강남구 봉은사로 지하 102,37.505581,127.024277,2017-06-13,10.0,NaN,LCD
3,2303,논현역 7번출구,강남구,서울특별시 강남구 학동로 지하 102,37.511517,127.021477,2017-06-13,15.0,NaN,LCD
4,2304,신영 ROYAL PALACE 앞,강남구,서울특별시 강남구 언주로 626,37.512527,127.035835,2017-06-13,10.0,NaN,LCD


In [9]:
place.dtypes

대여소번호          int64
보관소(대여소)명     object
자치구           object
상세주소          object
위도           float64
경도           float64
설치시기          object
LCD(거치대수)    float64
QR(거치대수)     float64
운영방식          object
dtype: object

In [13]:
place['설치시기']=place['설치시기'].apply(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d'))

In [22]:
place_1=place[place['설치시기']<'2020-04-01']

In [23]:
place_1

,대여소번호,보관소(대여소)명,자치구,상세주소,위도,경도,설치시기,LCD(거치대수),QR(거치대수),운영방식
1,2301,현대고등학교 건너편,강남구,서울특별시 강남구 압구정로 134,37.524071,127.021790,2017-06-13,10.0,NaN,LCD
2,2302,교보타워 버스정류장(신논현역 3번출구 후면),강남구,서울특별시 강남구 봉은사로 지하 102,37.505581,127.024277,2017-06-13,10.0,NaN,LCD
3,2303,논현역 7번출구,강남구,서울특별시 강남구 학동로 지하 102,37.511517,127.021477,2017-06-13,15.0,NaN,LCD
4,2304,신영 ROYAL PALACE 앞,강남구,서울특별시 강남구 언주로 626,37.512527,127.035835,2017-06-13,10.0,NaN,LCD
5,2305,MCM 본사 직영점 앞,강남구,서울특별시 강남구 언주로 734,37.520641,127.034508,2017-06-13,10.0,NaN,LCD
...,...,...,...,...,...,...,...,...,...,...
2033,1456,상아빌딩(우림시장 교차로),중랑구,서울특별시 중랑구 용마산로 494,37.595112,127.100327,2018-02-26,10.0,NaN,LCD
2034,1457,동원사거리,중랑구,서울특별시 중랑구 겸재로 229,37.589760,127.093239,2018-02-26,10.0,NaN,LCD
2035,1458,상봉터미널2,중랑구,망우동 534-58,37.597340,127.093086,2018-11-02,9.0,NaN,LCD
2036,1459,용마한신아파트사거리,중랑구,면목동 1086-79,37.597340,127.093086,2018-10-29,10.0,NaN,LCD


In [33]:
split_data=train_test_split(place_1, test_size=0.2, random_state=3)

In [35]:
train_place=split_data[0]
test_place=split_data[1]

In [36]:
train_place

,대여소번호,보관소(대여소)명,자치구,상세주소,위도,경도,설치시기,LCD(거치대수),QR(거치대수),운영방식
1324,1339,삼성전자서비스 성북센터,성북구,서울특별시 성북구 월계로 28,37.610569,127.033508,2017-11-13,9.0,NaN,LCD
1222,3511,응봉역 1번출구,성동구,서울특별시 성동구 고산자로 123,37.551250,127.035103,2017-11-01,10.0,NaN,LCD
115,2428,롯데리아 개포점앞,강남구,강남구 선릉로26,37.482368,127.061188,2020-02-18,NaN,10.0,QR
1488,2643,풍납중학교 측면,송파구,풍납동460,37.525291,127.112473,2020-01-21,NaN,10.0,QR
1061,178,증산3교 앞,서대문구,서울특별시 서대문구 북가좌동 352-9,37.579876,126.906349,2016-07-06,10.0,NaN,LCD
...,...,...,...,...,...,...,...,...,...,...
1052,166,가재울 초등학교,서대문구,서울특별시 서대문구 연희동 530,37.573277,126.919678,2016-06-29,20.0,NaN,LCD
1036,133,해담는다리,서대문구,서울특별시 서대문구 불광천길 150,37.582031,126.908997,2015-09-07,15.0,NaN,LCD
1825,303,광화문역 1번출구 앞,종로구,서울특별시 종로구 세종대로 지하189,37.571770,126.974663,2015-10-07,8.0,NaN,LCD
1400,1232,롯데마트 주차장 옆,송파구,서울특별시 송파구 석촌호수로12길 3-19,37.512089,127.096191,2017-06-22,10.0,NaN,LCD


In [37]:
test_place

,대여소번호,보관소(대여소)명,자치구,상세주소,위도,경도,설치시기,LCD(거치대수),QR(거치대수),운영방식
741,2917,신창중학교 후문 맞은편 버스정류장,노원구,서울 노원구 월계로45길 92,37.628071,127.046921,2020-03-30,NaN,10.0,QR
1626,250,대림아파트 사거리,영등포구,서울특별시 영등포구 신길동 249,37.507641,126.923080,2016-06-29,10.0,NaN,LCD
738,2913,상계8동 주민센터,노원구,서울시 노원구 동일로227길 85,37.666996,127.051529,2020-03-23,NaN,15.0,QR
1755,912,응암오거리,은평구,서울특별시 은평구 응암동 604-5,37.589661,126.916946,2016-09-02,10.0,NaN,LCD
490,574,아차산역4번출구,광진구,서울특별시 광진구 능동로 216,37.551849,127.088982,2016-07-06,40.0,NaN,LCD
...,...,...,...,...,...,...,...,...,...,...
373,2741,마곡수명산파크5-6단지,강서구,강서구 내발산동 751-1,37.550617,126.826881,2020-03-17,NaN,8.0,QR
1766,925,불광역 2번출구,은평구,서울특별시 은평구 불광동 285-43,37.609566,126.930977,2016-09-02,8.0,NaN,LCD
1795,961,불광역 9번 출구,은평구,불광동 281-109,37.610970,126.929810,2018-10-18,8.0,NaN,LCD
49,2352,서울본부세관(건설회관 앞),강남구,서울특별시 강남구 언주로 721,37.517590,127.035027,2017-08-22,10.0,NaN,LCD


In [39]:
place_1.to_csv('4월 이전 대여소.csv',encoding='cp949')

In [45]:
place[place['대여소번호']==1067]

,대여소번호,보관소(대여소)명,자치구,상세주소,위도,경도,설치시기,LCD(거치대수),QR(거치대수),운영방식
180,1067,한영중고앞사거리,강동구,명일동 59,37.549053,127.156403,2020-01-15,NaN,20.0,QR
